# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol)

#500 (generic function with 1 method)

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846900922764                   -0.70    4.0
  2   -7.852331226477       -2.27       -1.53    1.0
  3   -7.852646018876       -3.50       -2.54    3.0
  4   -7.852646676345       -6.18       -3.35    2.5
  5   -7.852646685809       -8.02       -4.74    1.2
  6   -7.852646686726       -9.04       -5.20    4.2
  7   -7.852646686729      -11.42       -5.81    1.8
  8   -7.852646686730      -12.31       -7.21    2.0
  9   -7.852646686730      -14.35       -7.59    2.0
 10   -7.852646686730   +  -15.05       -8.47    2.8
 11   -7.852646686730   +  -14.75       -9.80    3.2
 12   -7.852646686730   +    -Inf      -10.50    2.0
 13   -7.852646686730      -14.45      -11.40    2.2
 14   -7.852646686730   +  -14.75      -11.65    1.2
 15   -7.852646686730   +  -14.75      -12.48    2.8


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged)

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846854991847                   -0.70           4.2
  2   -7.852526639583       -2.25       -1.62   0.80    5.0
  3   -7.852610050354       -4.08       -2.79   0.80    5.2
  4   -7.852646569224       -4.44       -3.56   0.80    7.2
  5   -7.852646683544       -6.94       -4.51   0.80    8.8
  6   -7.852646686681       -8.50       -5.07   0.80   10.2
  7   -7.852646686725      -10.35       -6.12   0.80   11.5
  8   -7.852646686730      -11.31       -7.08   0.80   13.5
  9   -7.852646686730      -13.33       -7.76   0.80   15.5
 10   -7.852646686730   +    -Inf       -9.27   0.80   17.0
 11   -7.852646686730      -14.75       -9.28   0.80   19.0
 12   -7.852646686730   +    -Inf      -10.29   0.80   19.0
 13   -7.852646686730      -15.05      -11.32   0.80   20.8
 14   -7.852646686730   +  -14.45      -11.89   0.80   22.5
 15   -7.852646686730      -14.21      -

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), DFTK.RealFourierOperator[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.NonlocalOpera

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol)

# Newton algorithm

Iter     Function value   Gradient norm 
     0     1.391770e+01     3.215504e+00
 * time: 0.41474294662475586
     1     1.173411e+00     2.036530e+00
 * time: 0.6425089836120605
     2    -1.910784e+00     2.495047e+00
 * time: 0.6642529964447021
     3    -3.852037e+00     2.070376e+00
 * time: 0.6971609592437744
     4    -5.192825e+00     1.914682e+00
 * time: 0.7287590503692627
     5    -6.810036e+00     1.024074e+00
 * time: 0.7601509094238281
     6    -7.510892e+00     3.678687e-01
 * time: 0.7915830612182617
     7    -7.703285e+00     1.444654e-01
 * time: 0.8134100437164307
     8    -7.774616e+00     1.384072e-01
 * time: 0.835258960723877
     9    -7.817010e+00     8.976873e-02
 * time: 0.8572750091552734
    10    -7.837519e+00     6.544649e-02
 * time: 0.9489850997924805
    11    -7.846660e+00     4.031229e-02
 * time: 0.9709069728851318
    12    -7.850576e+00     3.557817e-02
 * time: 0.9928629398345947
    13    -7.851945e+00     1.238800e-02
 * time: 1.0146279335

(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1)

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846623650893                   -0.70    4.0
  2   -7.852288194554       -2.25       -1.53    1.0


(ham = Hamiltonian(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), HamiltonianBlock[DFTK.DftHamiltonianBlock(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), Any[DFTK.FourierMultiplication{Float64, Vector{Float64}}(PlaneWaveBasis(model = Model(lda_x+lda_c_pw, spin_polarization = :none), Ecut = 5.0 Ha, kgrid = [3, 3, 3]), KPoint([     0,      0,      0], spin = 1, num. G vectors =   137), [0.0, 0.5625437115174675, 2.25017484606987, 2.25017484606987, 0.5625437115174675, 0.5625437115174675, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276  …  3.7502914101164504, 3.7502914101164504, 2.0626602755640477, 1.5001165640465801, 0.7500582820232901, 2.0626602755640477, 4.50034969213974, 3.5627768396106276, 1.5001165640465801, 0.5625437115174675]), DFTK.RealSpaceMultiplication{Float64, A

Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol)

# Comparison of results

println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686156                   -2.55
  2   -7.852646686730       -9.24       -5.43
  3   -7.852646686730   +  -15.05      -10.33
  4   -7.852646686730   +    -Inf      -15.92
|ρ_newton - ρ_scf|  = 1.2711321000731736e-13
|ρ_newton - ρ_scfv| = 8.364214468349283e-14
|ρ_newton - ρ_dm|   = 1.7386016667545113e-9
